In [1]:
import pandas as pd
import sqlite3
import xml.etree.ElementTree as ET
from lxml import etree

### Datatranformatie van xml in SQLite3

In [ ]:
def process_xml_to_sqlite(xml_path, db_path, tabel_naam, xml_tag, kolom_definities, kolom_pad, batch_size=100):

    # Connect to SQLite 
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("PRAGMA journal_mode = WAL;")
    cursor.execute("PRAGMA synchronous = OFF;")
    
    # Verwijder de tabel als het al bestaat en vervang met een nieuwe tabel
    cursor.execute(f"DROP TABLE IF EXISTS {tabel_naam}")
    cursor.execute(f"CREATE TABLE {tabel_naam} ({kolom_definities})")
    conn.commit()

    data = []
    # Verwerking van de XML 
    context = etree.iterparse(xml_path, events=("end",), tag=xml_tag)
    for event, elem in context:
        row = tuple(elem.findtext(path) for path in kolom_pad)
        data.append(row)
        
        # bulk insert als de batch hoeveelheid bereikt is
        if len(data) >= batch_size:
            placeholders = ", ".join("?" for _ in kolom_pad)
            cursor.executemany(
                f"INSERT OR REPLACE INTO {tabel_naam} VALUES ({placeholders})", data
            )
            conn.commit()
            data.clear()
        
        # Vrijmaken van geheugen door verwerkte elementen te verwijderen
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]

    # Insert any remaining rows
    if data:
        placeholders = ", ".join("?" for _ in kolom_pad)
        cursor.executemany(
            f"INSERT OR REPLACE INTO {tabel_naam} VALUES ({placeholders})", data
        )
        conn.commit()
    
    conn.close()
    print(f"XML is succesvol opgeslagen in SQLite: {db_path} (Tabel: {tabel_naam})")

#### archeologischOnderzoeksgebied

In [ ]:
# archeologischOnderzoeksgebied
kolom_definities_archeologischOnderzoeksgebied = """
    choId TEXT PRIMARY KEY,
    typeCho TEXT,
    vertrouwelijkheidPreflabel TEXT,
    vertrouwelijkheidUri TEXT,
    objectnummer TEXT,
    basisregistratieId TEXT,
    bagRelatieId TEXT,
    plaatsnaam TEXT,
    geometrieId TEXT,
    besluit TEXT,
    WKTRD TEXT,
    WKT TEXT,
    registratiedatum TEXT
"""
kolom_pad_archeologischOnderzoeksgebied = (
    "choId",
    "typeCho",
    "vertrouwelijkheid/preflabel",
    "vertrouwelijkheid/uri",
    "objectnummer",
    "basisregistratie/basisregistratieId",
    "basisregistratie/bagRelatie/bagRelatieId",
    "asisregistratie/bagRelatie/plaatsnaam",
    "geometrie/geometrieId",
    "geometrie/besluit",
    "geometrie/WKTRD",
    "geometrie/WKT",
    "registratiedatum"
)

#### archeologischComplex

In [ ]:
# archeologischComplex
kolom_definities_archeologischComplex = """
    choId TEXT PRIMARY KEY,
    typeCho TEXT,
    vertrouwelijkheidPreflabel TEXT,
    vertrouwelijkheidUri TEXT,
    objectnummer TEXT,
    identificatie TEXT,
    samenhangChoId TEXT,
    samenhangTypeCho TEXT,
    samenhangRelatiePreflabel TEXT,
    samenhangRelatieUri TEXT,
    samenhangToelichting TEXT,
    kennisregistratienummer TEXT,
    typeKennis TEXT,
    zaakId TEXT,
    kennisVertrouwelijkheidPreflabel TEXT,
    kennisVertrouwelijkheidUri TEXT,
    formeelStandpunt TEXT,
    typekenmerkPreflabel TEXT,
    typekenmerkUri TEXT,
    typenaamPreflabel TEXT,
    typenaamUri TEXT
"""
kolom_pad_archeologischComplex = (
    "choId",
    "typeCho",
    "vertrouwelijkheid/preflabel",
    "vertrouwelijkheid/uri",
    "objectnummer",
    "identificatie",
    "samenhang/isRelatieVan/choId",
    "samenhang/isRelatieVan/typeCho",
    "samenhang/relatieType/preflabel",
    "samenhang/relatieType/uri",
    "samenhang/toelichting",
    "kennisregistratie/kennisregistratienummer",
    "kennisregistratie/typeKennis",
    "kennisregistratie/zaakId",
    "kennisregistratie/vertrouwelijkheid/preflabel",
    "kennisregistratie/vertrouwelijkheid/uri",
    "kennisregistratie/formeelStandpunt",
    "kennisregistratie/type/typekenmerk/preflabel",
    "kennisregistratie/type/typekenmerk/uri",
    "kennisregistratie/type/typenaam/preflabel",
    "kennisregistratie/type/typenaam/uri"
)

#### grondsporen

In [ ]:

kolom_definities_grondsporen = """
    choId TEXT NOT NULL,
    typeCho TEXT,
    objectnummer TEXT,
    identificatie TEXT,
    kennisregistratienummer TEXT,
    typeKennis TEXT,
    formeelStandpunt TEXT,
    omschrijving TEXT,
    omschrijvingPreflabel TEXT,
    omschrijvingUri TEXT,
    registratiedatum TEXT
"""

kolom_pad_grondsporen = (
    "choId",
    "typeCho",
    "objectnummer",
    "identificatie",
    "kennisregistratie/kennisregistratienummer",
    "kennisregistratie/typeKennis",
    "kennisregistratie/formeelStandpunt",
    "kennisregistratie/omschrijving/tekst",
    "kennisregistratie/omschrijving/omschrijvingstype/preflabel",
    "kennisregistratie/omschrijving/omschrijvingstype/uri",
    "registratiedatum"
)


#### vondstlocatie

In [ ]:
kolom_definities_vondstlocatie = """
    choId TEXT NOT NULL,
    typeCho TEXT,
    vertrouwelijkheidPreflabel TEXT,
    vertrouwelijkheidUri TEXT,
    objectnummer TEXT,
    basisregistratieId TEXT,
    bagRelatieId TEXT,
    plaatsnaam TEXT,
    locatieAanduidingId TEXT,
    locatietype TEXT,
    locatienaam TEXT,
    kennisregistratienummer TEXT,
    typeKennis TEXT,
    zaakId TEXT,
    kennisVertrouwelijkheidPreflabel TEXT,
    kennisVertrouwelijkheidUri TEXT,
    formeelStandpunt TEXT,
    omschrijving TEXT,
    omschrijvingPreflabel TEXT,
    omschrijvingUri TEXT,
    registratiedatum TEXT
"""

kolom_pad_vondstlocatie = (
    "choId",
    "typeCho",
    "vertrouwelijkheid/preflabel",
    "vertrouwelijkheid/uri",
    "objectnummer",
    "basisregistratie/basisregistratieId",
    "basisregistratie/bagRelatie/bagRelatieId",
    "basisregistratie/bagRelatie/plaatsnaam",
    "locatieAanduiding/locatieAanduidingId",
    "locatieAanduiding/locatietype",
    "locatieAanduiding/locatienaam",
    "kennisregistratie/kennisregistratienummer",
    "kennisregistratie/typeKennis",
    "kennisregistratie/zaakId",
    "kennisregistratie/vertrouwelijkheid/preflabel",
    "kennisregistratie/vertrouwelijkheid/uri",
    "kennisregistratie/formeelStandpunt",
    "kennisregistratie/omschrijving/tekst",
    "kennisregistratie/omschrijving/omschrijvingstype/preflabel",
    "kennisregistratie/omschrijving/omschrijvingstype/uri",
    "registratiedatum"
)


#### Vondsten

In [ ]:
kolom_definities_vondsten = """
    choId TEXT PRIMARY KEY,
    typeCho TEXT,
    vertrouwelijkheidPreflabel TEXT,
    vertrouwelijkheidUri TEXT,
    objectnummer TEXT,
    identificatie TEXT,
    samenhangChoId TEXT,
    samenhangTypeCho TEXT,
    samenhangRelatiePreflabel TEXT,
    samenhangRelatieUri TEXT,
    samenhangToelichting TEXT,
    kennisregistratienummer TEXT,
    typeKennis TEXT,
    zaakId TEXT,
    kennisVertrouwelijkheidPreflabel TEXT,
    kennisVertrouwelijkheidUri TEXT,
    formeelStandpunt TEXT,
    typekenmerkPreflabel TEXT,
    typekenmerkUri TEXT,
    typennaamPreflabel TEXT,
    typennaamUri TEXT,
    aantal TEXT,
    outerRegistratiedatum TEXT
"""

kolom_pad_vondsten = (
    "choId",
    "typeCho",
    "vertrouwelijkheid/preflabel",
    "vertrouwelijkheid/uri",
    "objectnummer",
    "identificatie",
    "samenhang/isRelatieVan/choId",
    "samenhang/isRelatieVan/typeCho",
    "samenhang/relatieType/preflabel",
    "samenhang/relatieType/uri",
    "samenhang/toelichting",
    "kennisregistratie[typeKennis='Type']/kennisregistratienummer",
    "kennisregistratie[typeKennis='Type']/typeKennis",
    "kennisregistratie[typeKennis='Type']/zaakId",
    "kennisregistratie[typeKennis='Type']/vertrouwelijkheid/preflabel",
    "kennisregistratie[typeKennis='Type']/vertrouwelijkheid/uri",
    "kennisregistratie[typeKennis='Type']/formeelStandpunt",
    "kennisregistratie[typeKennis='Type']/type/typekenmerk/preflabel",
    "kennisregistratie[typeKennis='Type']/type/typekenmerk/uri",
    "kennisregistratie[typeKennis='Type']/type/typenaam/preflabel",
    "kennisregistratie[typeKennis='Type']/type/typenaam/uri",
    "aantal",
    "registratiedatum"
)

#### Datasets naar database

In [ ]:
# Archeologisch onderzoeksgebied
process_xml_to_sqlite("sourceDatasets/ldv_cho_archis.xml", "database.db", "archeologischOnderzoeksgebiedPostTriply",
                       "archeologischOnderzoeksgebied", kolom_definities_archeologischOnderzoeksgebied, kolom_pad_archeologischOnderzoeksgebied)

In [ ]:
# archeologischComplex
process_xml_to_sqlite("sourceDatasets/ldv_cho_archis.xml", "database.db", "archeologischComplexPostTriply",
                       "archeologischComplex", kolom_definities_archeologischComplex, kolom_pad_archeologischComplex)

In [ ]:
# vondsten
process_xml_to_sqlite(
    xml_path="sourceDatasets/ldv_cho_archis.xml",
    db_path="database.db",
    tabel_naam="vondstenPostTriply",
    xml_tag="vondsten",  
    kolom_definities=kolom_definities_vondsten,
    kolom_pad=kolom_pad_vondsten,
    batch_size=100
)


In [ ]:
# vondstlocatie
process_xml_to_sqlite(
        xml_path="sourceDatasets/ldv_cho_archis.xml",
        db_path="database.db",
        tabel_naam="vondstlocatiePostTriply",
        xml_tag="vondstlocatie",
        kolom_definities=kolom_definities_vondstlocatie,
        kolom_pad=kolom_pad_vondstlocatie,
        batch_size=100
    )

In [ ]:
# grondsporen
process_xml_to_sqlite(
    xml_path="sourceDatasets/ldv_cho_archis.xml",
    db_path="database.db",
    tabel_naam="grondsporenPostTriply",
    xml_tag="grondsporen",
    kolom_definities=kolom_definities_grondsporen,
    kolom_pad=kolom_pad_grondsporen,
    batch_size=100
)

### Dataverwerking

In [ ]:
def load_table_to_dataframe(db_path, tabel_naam):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(f"SELECT * FROM {tabel_naam}", conn)
    conn.close()
    return df

In [ ]:
df_archeologischOnderzoeksgebiedPostTriply = load_table_to_dataframe("database.db", "archeologischOnderzoeksgebiedPostTriply")
df_archeologischComplexPostTriply = load_table_to_dataframe("database.db", "archeologischComplexPostTriply")
df_vondstenPostTriply = load_table_to_dataframe("database.db", "vondstenPostTriply")
df_vondstlocatiePostTriply = load_table_to_dataframe("database.db", "vondstlocatiePostTriply")
df_grondsporenPostTriply = load_table_to_dataframe("database.db", "grondsporenPostTriply")

In [11]:
df_postTriply = pd.concat([df_archeologischOnderzoeksgebiedPostTriply, 
                           df_archeologischComplexPostTriply, 
                           df_vondstenPostTriply, 
                           df_vondstlocatiePostTriply, 
                           df_grondsporenPostTriply])

In [12]:
df_postTriply.to_csv('csv/postTriplyDataframe.csv', index=False)

In [13]:
del df_archeologischOnderzoeksgebiedPostTriply, df_archeologischComplexPostTriply, df_vondstenPostTriply, df_vondstlocatiePostTriply, df_grondsporenPostTriply